# CRIM Intervals:  Modules

### What You Can Do with this Notebook

* Find contrapuntal modules, which are ngrams representing the combination of melodic and harmonic intervals made by every pair of voices in a piece
* Count and filter these modules
* Search for them in one piece or an entire corpus.



### A. Import Intervals and Other Code


In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import glob


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [14]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# and combine the strings and load the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei
{'title': 'Ave Maria', 'composer': 'Josquin Des Prés'}


## D. Corpus Inventory

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* See the Corpus Methods notebook for a full explanation of ways to import local and remote files


>`import glob
corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)
corpus = CorpusBase(corpus_list)`


In [7]:
number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        corpus_list.append(name)
        print(name  + 'has been added to your list')
    else:
        print('sorry, ' + name  + 'is not a three-voice piece')

corpus = CorpusBase(corpus_list)  

Previously imported piece detected.
sorry, Music_Files/Morley_1597_132_06.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_132_07.musicxmlis not a three-voice piece
Previously imported piece detected.
Music_Files/Morley_1597_127_2.musicxmlhas been added to your list
Previously imported piece detected.
Music_Files/Morley_1597_127_3.musicxmlhas been added to your list
Previously imported piece detected.
sorry, Music_Files/Morley_1597_131_4.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_131_5.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_074_01.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_133_04.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_133_05.musicxmlis not a three-voice piece
Previously imported piece detected.

sorry, Music_Files/Morley_1597_190-193.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_134_1.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_135_8.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_075_1.musicxmlis not a three-voice piece
Previously imported piece detected.
Music_Files/Morley_1597_128_05.musicxmlhas been added to your list
Previously imported piece detected.
Music_Files/Morley_1597_128_04.musicxmlhas been added to your list
Previously imported piece detected.
sorry, Music_Files/Morley_1597_135_2.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_135_3.musicxmlis not a three-voice piece
Previously imported piece detected.
Music_Files/Morley_1597_129_2.musicxmlhas been added to your list
Previously imported piece detected.
Music_Files/Morley_1597_129_3.musicxmlhas 

### D.1  Corpus Ngram Inventory

*  Get the `ngrams` for all of them.  
*  In this case:  modules of length "3", with diatonic simple intervals
*  Then combine them into one frame



* NB: use `ImportedPiece`, not `piece`!
* NB:  for `func` do **NOT** include the closing parentheses!

    - `func = ImportedPiece.ngrams`
    - `list_of_modules = corpus.batch(func=func, kwargs={'n': 3, 'interval_settings': ('d', False, True)}, metadata=True)`

    - `title_of_output = pd.concat(list_of_melodic_ngrams)`

* Note that the output lists voice pairs by **staff position**.  In a four-voice piece, the lowest part would be 4, then 3, etc.  `4_3` represents the two lowest voices of the piece, and so on.


In [5]:
# remember to omit "()" after calling `ngrams`
# here we are using simple diatonic
func = ImportedPiece.ngrams
list_of_modules = corpus.batch(func=func, kwargs={'n': 3, 'interval_settings': ('d', True, True), 'offsets': 'last'}, metadata=True)
func2 = ImportedPiece.detailIndex
list_of_details = corpus.batch(func=func2, kwargs={'offset': True, 'df': list_of_modules})

module_corpus = pd.concat(list_of_details).fillna('-').dropna(how="all")

c = module_corpus['Composer']
t = module_corpus["Title"]
module_corpus.pop("Composer")
module_corpus.pop("Title")
module_corpus.insert(0, "Composer", c)
module_corpus.insert(1, "Title", t)
module_corpus

Composer                Title                4_3  \
Measure Beat Offset                                                    
1       3.5  5.0      Morley   Morley_1597_132_06                  -   
        4.0  6.0      Morley   Morley_1597_132_06       5_-2, 3_2, 3   
2       1.0  8.0      Morley   Morley_1597_132_06       3_2, 3_-4, 5   
        2.0  10.0     Morley   Morley_1597_132_06                  -   
        3.0  12.0     Morley   Morley_1597_132_06       3_-4, 5_4, 3   
...                      ...                  ...                ...   
50      1.0  196.0    Morley  Morley_1597_188-189     5_Held, 6_2, 3   
        2.0  197.0    Morley  Morley_1597_188-189     6_2, 3_Held, 4   
        3.0  198.0    Morley  Morley_1597_188-189                  -   
        4.0  199.0    Morley  Morley_1597_188-189  3_Held, 4_Held, 3   
51      1.0  200.0    Morley  Morley_1597_188-189    4_Held, 3_-5, 8   

                                  4_2                4_1           3_2  \
Measure Beat Offset                                                      
1       3.5  5.0                    -  10_-2, 10_Held, 9             -   
        4.0  6.0         8_-2, 6_2, 5    10_Held, 9_2, 8  4_-4, 4_2, 3   
2       1.0  8.0         6_2, 5_-4, 8      9_2, 8_-4, 11  4_2, 3_-2, 4   
        2.0  10.0                   -  8_-4, 11_Held, 10             -   
        3.0  12.0        5_-4, 8_4, 5   11_Held, 10_4, 8  3_-2, 4_2, 3   
...                               ...                ...           ...   
50      1.0  196.0   10_Held, 10_2, 8     8_Held, 8_2, 7  6_2, 5_-3, 6   
        2.0  197.0                  -     8_2, 7_Held, 6  5_-3, 6_2, 5   
        3.0  198.0                  -  7_Held, 6_Held, 5             -   
        4.0  199.0                  -                  -  6_2, 5_-2, 6   
51      1.0  200.0     10_2, 8_-5, 12   6_Held, 5_-5, 10  5_-2, 6_2, 5   

                                 3_1                   2_1 5_4 5_3 5_2 5_1  
Measure Beat Offset                                                         
1       3.5  5.0     6_-4, 8_Held, 7       3_-4, 5_Held, 4   -   -   -   -  
        4.0  6.0      8_Held, 7_2, 6        5_Held, 4_1, 4   -   -   -   -  
2       1.0  8.0        7_2, 6_-2, 7           4_1, 4_1, 4   -   -   -   -  
        2.0  10.0    6_-2, 7_Held, 6        4_1, 4_Held, 3   -   -   -   -  
        3.0  12.0     7_Held, 6_2, 6        4_Held, 3_1, 4   -   -   -   -  
...                              ...                   ...  ..  ..  ..  ..  
50      1.0  196.0      4_2, 3_-3, 5       -3_1, -3_-2, -2   -   -   -   -  
        2.0  197.0      3_-3, 5_2, 3    -3_-2, -2_Held, -3   -   -   -   -  
        3.0  198.0    5_2, 3_Held, 2  -2_Held, -3_Held, -4   -   -   -   -  
        4.0  199.0   3_Held, 2_-2, 3                     -   -   -   -   -  
51      1.0  200.0      2_-2, 3_2, 3     -3_Held, -4_1, -3   -   -   -   -  

[5280 rows x 12 columns]

In [8]:
# Modules for a selected piece, pulled from above results
# Saved to CSV

Morley_1597_194 = module_corpus[module_corpus['Title'].isin(['Morley_1597_194'])]
Morley_1597_194.to_csv('Morley_1597_194.csv')

### D.2  Search Ngrams in the Corpus

* Note that the interval and pattern settings (diatonic/chromatic; compound/simple; length of ngrams) for this tool are set in the request for **module_corpus** above.

In [6]:
# HERE we have repeating modules from the Josquin opening PEN
@interact
def new_ngram_search(my_search="", df = fixed(module_corpus)):
    df2 = module_corpus.copy()
    c = df['Composer']
    t = df["Title"]
    df2.pop("Composer")
    df2.pop("Title")
    df2.insert(0, "Composer", c)
    df2.insert(1, "Title", t)
    filtered_ngrams = df2[df2.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))